In [55]:
import pandas as pd
import sys
import numpy as np
from dotenv import load_dotenv
load_dotenv()
import os
import requests
from google_maps_reviews import ReviewsClient
api_client = ReviewsClient(api_key=os.getenv("OUTSCRAPE_NEW_KEY"))

In [56]:
data = pd.read_csv("Data/companies_with_address.csv" , delimiter=";")


In [57]:
def get_place_id(name, address, place):
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": f"{name}, {address} {place}",
        "inputtype": "textquery",
        "fields": "place_id",
        "key": os.getenv("GOOGLE_API_KEY")
    }

    response = requests.get(base_url, params=params)
    result = response.json()

    if result.get("candidates"):
        return result["candidates"][0]["place_id"]
    else:
        return None

In [58]:
data['place_id'] = data.head(10).apply(lambda row: get_place_id(row['Naam'], row['Adres'], row['Gemeente']), axis=1)
data.head()

,Naam,Gemeente,Laatste jaar,Adres,Ondernemingsnummer,BvD ID nummer,place_id
0,ALPHA CREDIT,BRUSSEL,2023.0,Warandeberg 8 b.3,0445 781 316,BE0445781316,ChIJH-_EZIDEw0cR0R-3VfmWrSk
1,COWBOY,BRUSSEL,2023.0,Regentschapsstraat 67,0669 908 031,BE0669908031,ChIJWysouY7Fw0cRO7FXCva-I-8
2,DOD,BRUSSEL,2014.0,Leuvensesteenweg 16,0419 684 455,BE0419684455,ChIJRx4JfGPDw0cRFKkBYiz9EZQ
3,UNIVERSAL TRADING SOLUTIONS,BRUSSEL,2013.0,Louizalaan 367,0849 251 826,BE0849251826,ChIJCSJGeu7Ew0cRUXzNiq8nekU
4,VANBO,BEVEREN-LEIE,2011.0,Grote Heerweg 69,0426 734 474,BE0426734474,ChIJU_xXtAU8w0cRvnFySR_t8Bw


In [ ]:
data['reviews'] = data.head(10).apply(lambda row: api_client.get_reviews(row['place_id'], 250, language='en'), axis=1)

data.head(10)

,Naam,Gemeente,Laatste jaar,Adres,Ondernemingsnummer,BvD ID nummer,place_id,reviews
0,ALPHA CREDIT,BRUSSEL,2023.0,Warandeberg 8 b.3,0445 781 316,BE0445781316,ChIJH-_EZIDEw0cR0R-3VfmWrSk,"[{'query': 'ChIJH-_EZIDEw0cR0R-3VfmWrSk', 'nam..."
1,COWBOY,BRUSSEL,2023.0,Regentschapsstraat 67,0669 908 031,BE0669908031,ChIJWysouY7Fw0cRO7FXCva-I-8,"[{'query': 'ChIJWysouY7Fw0cRO7FXCva-I-8', 'nam..."
2,DOD,BRUSSEL,2014.0,Leuvensesteenweg 16,0419 684 455,BE0419684455,ChIJRx4JfGPDw0cRFKkBYiz9EZQ,"[{'query': 'ChIJRx4JfGPDw0cRFKkBYiz9EZQ', 'nam..."
3,UNIVERSAL TRADING SOLUTIONS,BRUSSEL,2013.0,Louizalaan 367,0849 251 826,BE0849251826,ChIJCSJGeu7Ew0cRUXzNiq8nekU,"[{'query': 'ChIJCSJGeu7Ew0cRUXzNiq8nekU', 'nam..."
4,VANBO,BEVEREN-LEIE,2011.0,Grote Heerweg 69,0426 734 474,BE0426734474,ChIJU_xXtAU8w0cRvnFySR_t8Bw,"[{'query': 'ChIJU_xXtAU8w0cRvnFySR_t8Bw', 'nam..."
5,DH-GROUP,LOKEREN,2015.0,Waaslandlaan 3,0456 843 571,BE0456843571,ChIJ6_cbCTCFw0cRXBIM8wAOKb4,"[{'query': 'ChIJ6_cbCTCFw0cRXBIM8wAOKb4', 'nam..."
6,COMPUTER SERVICES SOLUTIONS,ZAVENTEM,2006.0,Excelsiorlaan 75,0425 645 007,BE0425645007,ChIJczt_fC_cw0cR_kWjibB-6i0,"[{'query': 'ChIJczt_fC_cw0cR_kWjibB-6i0', 'nam..."
7,JOHAN'S MACHINERY,DADIZELE,2007.0,Meensesteenweg 206,0441 774 919,BE0441774919,ChIJLbzPUS0yw0cRIDVip2Dx3iM,"[{'query': 'ChIJLbzPUS0yw0cRIDVip2Dx3iM', 'nam..."
8,ECOPHARMA,OUDENAARDE,2013.0,Ambachtsstraat 1,0462 413 945,BE0462413945,ChIJKcYBTgASw0cRtRKrOZ7M4i4,"[{'query': 'ChIJKcYBTgASw0cRtRKrOZ7M4i4', 'nam..."
9,SUNNYLAND DISTRIBUTION,TURNHOUT,2014.0,Everdongenlaan 12,0461 326 159,BE0461326159,ChIJ5ecJXbOzxkcRK78GNwIMpEQ,"[{'query': 'ChIJ5ecJXbOzxkcRK78GNwIMpEQ', 'nam..."
